# 📌 Extracción

In [ ]:
import json
import pandas as pd

# Cargar archivo JSON
with open('/mnt/data/TelecomX_Data.json') as f:
    data = json.load(f)

# Normalizar los datos anidados
df = pd.json_normalize(data)

# Vista previa de los datos
df.head()

# 🔧 Transformación

In [ ]:
# Renombrar columnas para mayor claridad
df.columns = df.columns.str.replace(r'\.', '_')

# Convertir columnas numéricas que vienen como texto
df['account_Charges_Total'] = pd.to_numeric(df['account_Charges_Total'], errors='coerce')

# Verificar valores nulos
missing_values = df.isnull().sum()

# Rellenar o eliminar registros con datos faltantes si es necesario
df = df.dropna(subset=['Churn'])  # Por ejemplo, eliminar registros sin información de evasión

# Transformar la variable 'Churn' en binaria
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Convertir 'SeniorCitizen' a categórica
df['customer_SeniorCitizen'] = df['customer_SeniorCitizen'].map({1: 'Yes', 0: 'No'})

# Resultado de la limpieza
df.info()


# 📊 Carga y análisis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Proporción de evasión de clientes
churn_rate = df['Churn'].mean()
print(f"Tasa de evasión: {churn_rate:.2%}")

# Análisis de churn por tipo de contrato
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='account_Contract', hue='Churn')
plt.title('Churn por tipo de contrato')
plt.show()

# Análisis de churn por servicio de internet
plt.figure(figsize=(8, 5))
sns.countplot(data=df, x='internet_InternetService', hue='Churn')
plt.title('Churn por tipo de Internet')
plt.show()

# Distribución del total gastado según churn
plt.figure(figsize=(10, 6))
sns.boxplot(x='Churn', y='account_Charges_Total', data=df)
plt.title('Distribución del gasto total por churn')
plt.show()


# 🔄 Complemento: Análisis visual avanzado

🔹 1. Mapa de calor de correlación

In [ ]:
# Convertir todas las columnas categóricas relevantes a numéricas para el análisis de correlación
df_corr = df.copy()

# Selección de columnas categóricas a convertir
cat_cols = [
    'customer_gender', 'customer_SeniorCitizen', 'customer_Partner', 'customer_Dependents',
    'phone_PhoneService', 'phone_MultipleLines', 'internet_InternetService',
    'internet_OnlineSecurity', 'internet_OnlineBackup', 'internet_DeviceProtection',
    'internet_TechSupport', 'internet_StreamingTV', 'internet_StreamingMovies',
    'account_Contract', 'account_PaperlessBilling', 'account_PaymentMethod'
]

df_corr[cat_cols] = df_corr[cat_cols].astype('category').apply(lambda x: x.cat.codes)

# Correlación con Churn
plt.figure(figsize=(12, 10))
sns.heatmap(df_corr.corr(), cmap='coolwarm', annot=True, fmt=".2f", square=True)
plt.title('Mapa de Calor de Correlaciones')
plt.show()


🔹 2. Distribución de clientes por tiempo de contrato (tenure) y churn

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(data=df, x='customer_tenure', bins=30, hue='Churn', multiple='stack')
plt.title('Distribución de Tenure por Churn')
plt.xlabel('Meses de Antigüedad')
plt.ylabel('Número de Clientes')
plt.show()


🔹 3. Churn por cada servicio adicional

In [ ]:
services = [
    'internet_OnlineSecurity', 'internet_OnlineBackup', 'internet_DeviceProtection',
    'internet_TechSupport', 'internet_StreamingTV', 'internet_StreamingMovies'
]

for service in services:
    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x=service, hue='Churn')
    plt.title(f'Churn por {service.split("_")[1]}')
    plt.xticks(rotation=15)
    plt.show()


🔹 5. Clientes senior y con dependientes

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='customer_SeniorCitizen', hue='Churn')
plt.title('Churn por Clientes Senior')
plt.show()

plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='customer_Dependents', hue='Churn')
plt.title('Churn por Clientes con Dependientes')
plt.show()


# 📄Informe Preliminar

In [ ]:
# Cálculos clave
churn_rate = df['Churn'].mean() * 100

# Contrato
contract_churn = df.groupby('account_Contract')['Churn'].mean().sort_values(ascending=False) * 100

# InternetService
internet_churn = df.groupby('internet_InternetService')['Churn'].mean().sort_values(ascending=False) * 100

# Paperless Billing
paperless_churn = df.groupby('account_PaperlessBilling')['Churn'].mean() * 100

# Payment Method
payment_churn = df.groupby('account_PaymentMethod')['Churn'].mean().sort_values(ascending=False) * 100

# Tenure promedio según churn
tenure_churn = df.groupby('Churn')['customer_tenure'].mean()

# Monthly charges promedio según churn
monthly_charges = df.groupby('Churn')['account_Charges_Monthly'].mean()

print("🔍 Informe Final - Evasión de Clientes en Telecom X\n")
print(f"Tasa general de evasión: {churn_rate:.2f}%\n")

print("📌 Conclusiones principales basadas en los datos:\n")

print(f"1. Los contratos 'Month-to-month' presentan una tasa de churn de {contract_churn['Month-to-month']:.2f}%, muy superior a:")
print(f"   - 'One year': {contract_churn['One year']:.2f}%")
print(f"   - 'Two year': {contract_churn['Two year']:.2f}%")

print(f"\n2. Los clientes con servicio de Internet 'Fiber optic' tienen una tasa de churn de {internet_churn.get('Fiber optic', 0):.2f}%,")
print(f"   comparado con 'DSL': {internet_churn.get('DSL', 0):.2f}% y 'No': {internet_churn.get('No', 0):.2f}%")

print(f"\n3. La facturación electrónica ('Yes') presenta una evasión de {paperless_churn.get('Yes', 0):.2f}%,")
print(f"   mientras que la facturación tradicional ('No') tiene {paperless_churn.get('No', 0):.2f}%")

print(f"\n4. El método de pago con mayor churn es '{payment_churn.idxmax()}' con {payment_churn.max():.2f}%")

print(f"\n5. La antigüedad promedio (tenure) de clientes que abandonan es de {tenure_churn[1]:.1f} meses,")
print(f"   mientras que los que se quedan tienen un promedio de {tenure_churn[0]:.1f} meses")

print(f"\n6. Los clientes que se van tienen un cargo mensual promedio de ${monthly_charges[1]:.2f},")
print(f"   comparado con ${monthly_charges[0]:.2f} para los que permanecen.\n")

